Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in dow30.csv
dow30 = pd.read_csv('dow30.csv')

# Remove the exchange from the beginning of the symbol.
def _symbol(row):
    return row['Symbol'].split(':')[-1].strip()

dow30['Symbol'] = dow30.apply(_symbol, axis=1)
dow30.drop(columns=['Exchange', 'Date added', 'Notes', 'Index weighting'], inplace=True)
dow30.rename(columns={'Symbol': 'Symbol',
                      'Company':'Description',
                      'Industry':'Asset Class'}, inplace=True)

dow30.set_index("Symbol", inplace=True)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Biopharmaceutical
AAPL,Apple,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar,Construction and Mining
CVX,Chevron,Petroleum industry
CSCO,Cisco,Information technology
KO,Coca-Cola,Drink industry


In [6]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Conglomerate': 'US Stocks:Industrials',
 'Financial services': 'US Stocks:Financials',
 'Pharmaceutical industry': 'US Stocks:Healthcare',
 'Biopharmaceutical': 'US Stocks:Healthcare',
 'Information technology': 'US Stocks:Technology',
 'Aerospace and defense': 'US Stocks:Industrials',
 'Construction and Mining': 'US Stocks:Industrials',
 'Petroleum industry': 'US Stocks:Energy',
 'Food industry': 'US Stocks:Consumer Staples',
 'Drink industry': 'US Stocks:Consumer Staples',
 'Chemical industry': 'US Stocks:Materials',
 'Retailing': 'US Stocks:Consumer Discretionary',
 'Apparel': 'US Stocks:Consumer Discretionary',
 'Fast-moving consumer goods': 'US Stocks:Consumer Staples',
 'Managed health care': 'US Stocks:Healthcare',
 'Telecommunication': 'US Stocks:Communication Services',
 'Broadcasting and entertainment': 'US Stocks:Communication Services',
 'Home Improvement': 'US Stocks:Consumer Discretionary',
 'Semiconductor industry': 'US Stocks:Technology',
 'Clothing industry': 'US Sto

In [7]:
# Map dow30 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

dow30['Asset Class'] = dow30.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
dow30.index = dow30.index.str.replace('.', '-', regex=False)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,US Stocks:Industrials
AXP,American Express,US Stocks:Financials
AMGN,Amgen,US Stocks:Healthcare
AAPL,Apple,US Stocks:Technology
BA,Boeing,US Stocks:Industrials
CAT,Caterpillar,US Stocks:Industrials
CVX,Chevron,US Stocks:Energy
CSCO,Cisco,US Stocks:Technology
KO,Coca-Cola,US Stocks:Consumer Staples


In [8]:
# Make symbols list.
symbols = list(dow30.index)
#symbols

In [9]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

MMM AXP AMGN AAPL BA CAT CVX CSCO KO DIS DOW GS HD HON IBM INTC JNJ JPM MCD MRK MSFT NKE PG CRM TRV UNH VZ V WBA WMT 


In [10]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DIS,...,MSFT,NKE,PG,CRM,TRV,UNH,VZ,V,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,128.50,82.20,128.15,24.60,113.66,72.61,79.64,21.46,32.60,87.38,...,40.81,43.64,71.30,59.24,87.62,89.24,32.02,62.74,60.07,72.25
2015-01-05,125.60,80.03,126.63,23.91,112.87,68.78,76.46,21.03,32.60,86.10,...,40.44,42.93,70.97,58.17,86.56,87.77,31.75,61.36,58.88,72.04
2015-01-06,124.26,78.32,122.55,23.91,111.54,68.34,76.42,21.02,32.84,85.64,...,39.84,42.68,70.64,57.20,85.79,87.59,32.07,60.96,59.04,72.60
2015-01-07,125.16,80.03,126.83,24.25,113.27,69.40,76.36,21.22,33.25,86.52,...,40.35,43.56,71.01,56.93,87.25,88.49,31.86,61.78,60.54,74.52
2015-01-08,128.16,81.17,126.37,25.18,115.28,70.11,78.10,21.38,33.66,87.41,...,41.54,44.57,71.82,58.59,89.06,92.71,32.55,62.61,61.30,76.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-27,115.25,172.31,253.65,145.93,211.17,264.54,179.45,48.51,60.49,109.54,...,248.16,127.53,140.57,164.52,188.76,486.05,40.64,231.44,37.17,143.30
2023-01-30,112.58,172.66,251.59,143.00,209.67,261.50,174.20,48.22,60.64,107.59,...,242.71,126.37,141.01,164.75,187.91,485.79,41.05,229.10,36.53,142.15
2023-01-31,115.08,174.93,252.40,144.29,213.00,252.29,174.02,48.67,61.32,108.49,...,247.81,127.33,142.38,167.97,191.12,499.19,41.57,230.21,36.86,143.87


In [11]:
# Sample symbol.
symbol = 'MMM'

In [12]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

-25.108764981191044

In [13]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [14]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.3827212860552235

In [15]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.26361647626088913

In [16]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   127.12
2015-02-28   132.92
2015-03-31   130.00
2015-04-30   123.26
2015-05-31   126.17
2015-06-30   122.38
2015-07-31   120.03
2015-08-31   113.52
2015-09-30   113.22
2015-10-31   125.55
2015-11-30   125.87
2015-12-31   121.10
2016-01-31   121.39
2016-02-29   127.02
2016-03-31   134.92
2016-04-30   135.53
2016-05-31   137.19
2016-06-30   142.74
2016-07-31   145.38
2016-08-31   147.01
2016-09-30   144.54
2016-10-31   135.57
2016-11-30   141.76
2016-12-31   147.39
2017-01-31   144.30
2017-02-28   154.82
2017-03-31   158.96
2017-04-30   162.69
2017-05-31   170.89
2017-06-30   174.00
2017-07-31   168.13
2017-08-31   171.74
2017-09-30   176.43
2017-10-31   193.49
2017-11-30   205.40
2017-12-31   198.84
2018-01-31   211.62
2018-02-28   200.13
2018-03-31   186.54
2018-04-30   165.19
2018-05-31   168.75
2018-06-30   168.31
2018-07-31   181.66
2018-08-31   181.67
2018-09-30   181.49
2018-10-31   163.88
2018-11-30   180.31
2018-12-31   165.24
2019-01-31   173.70
2019-02-28   18

In [17]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     nan
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2022-10-31    0.14
2022-11-30    0.01
2022-12-31   -0.05
2023-01-31   -0.04
2023-02-28    0.05
Freq: M, Name: MMM, Length: 98, dtype: float64

In [18]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.23451063664105604

In [19]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: DOW 30 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [20]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

for i, (index, row) in enumerate(dow30.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [21]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')